# Model Replication

Making use of the classes in the "modules.py" (some classes modified from the mesa framework) and "Visu.py" files, results from replication and original model by Janssen and Rollins can be compared. Relational equiv?

In [3]:
from modules import Resource_Model, BatchRunner, Data_Collection
from Visu import Visu

## Exploring the Parameter Space: CPR Levels and Gini of Contributions / Extractions

Figs. 2, 3, and 4 in Janssen's and Rollins' paper show the averages of generated CPR level as well as Gini coefficients of contributions and extractions for different values for parameters alpha and beta. Setting up a model with Janssen's and Rollins' set of default parameter values (see Table 2 in Janssen's and Rollins' paper), the resulting graphs can be compared. First, the case for only asymmetric games is tested. The model is run for 50 iterations.

In [4]:
runs = 10
agents = 5
eta = 0.5
fa = 1
r = 2
iterations = 2
alpha_beta_values = [] #the range of values for alpha and beta with which the iterated model is run
for i in range(-10, 11):
    x = i/10
    alpha_beta_values.append(x)

Setting up mesa's BatchRunner for the iterated model, running the model, and plotting the output using matplotlib.

In [5]:
#parameters for the BatchRunner, i.e. the iterated model
param_values = {"N": agents, "alpha": alpha_beta_values, "beta": alpha_beta_values, "eta": eta,
                "fa": fa, "r": r}

model_reporter = {"Generated_CPR": Data_Collection.determine_final_resource,
                  "Gini_Contributions": Data_Collection.compute_gini_contributions, 
                  "Gini_Takeouts": Data_Collection.compute_gini_takeouts}
batch = BatchRunner(Resource_Model, param_values, iterations, runs, model_reporter)

batch.run_all()
out = batch.get_model_vars_dataframe() # Stores the collected data

visualize = Visu(out)
visualize.plot()